<a href="https://colab.research.google.com/github/uriamedalia/facial-expression-classifier/blob/split-notebooks/src/01_train_model_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Clone the GitHub repository
!git clone --branch split-notebooks https://github.com/uriamedalia/facial-expression-classifier.git

# Move into the project folder
%cd facial-expression-classifier/src

# Install dependencies
!pip install -q torch torchvision pandas numpy matplotlib seaborn scikit-learn pillow

# Check dataset structure
import os

train_path = "../data/train"
test_path = "../data/test"

print("✅ Train classes:", os.listdir(train_path))
print("✅ Test classes:", os.listdir(test_path))

Cloning into 'facial-expression-classifier'...
remote: Enumerating objects: 34197, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 34197 (delta 3), reused 8 (delta 1), pack-reused 34184 (from 3)
Receiving objects: 100% (34197/34197), 106.93 MiB | 12.98 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Updating files: 100% (35896/35896), done.
/content/facial-expression-classifier/src/facial-expression-classifier/src
✅ Train classes: ['disgust', 'neutral', 'sad', 'happy', 'angry', 'surprise', 'fear']
✅ Test classes: ['disgust', 'neutral', 'sad', 'happy', 'angry', 'surprise', 'fear']


# 01 - Train Model (Improved)

Fine-tunes a ResNet34 model on FER2013 with data augmentation and class weighting.

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from collections import Counter
import os

In [3]:
# Data augmentation for training set
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(48, padding=4),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Test transform only rescale and normalize
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dir = '../data/train'
test_dir = '../data/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# Model setup (ResNet34)
model = models.resnet34(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 7)

# Unfreeze last block
for param in model.layer4.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 158MB/s]


In [5]:
# Calculate class weights
targets = train_dataset.targets
counts = Counter(targets)
weights = torch.tensor([1.0 / counts[i] for i in range(7)], dtype=torch.float32)
weights = weights.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [8]:
# Training loop
num_epochs = 20
best_val_acc = 0.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)

    epoch_loss = train_loss / len(train_dataset)
    epoch_acc = train_corrects.double() / len(train_dataset)
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    val_loss /= len(test_dataset)
    val_acc = val_corrects.double() / len(test_dataset)
    print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '../models/resnet34_best_model.pth')
        print("✅ Best model saved.")

Epoch 1/20
Train Loss: 1.7218 Acc: 0.3474
Val   Loss: 1.5891 Acc: 0.3980
✅ Best model saved.
Epoch 2/20
Train Loss: 1.5621 Acc: 0.4139
Val   Loss: 1.6308 Acc: 0.4019
✅ Best model saved.
Epoch 3/20
Train Loss: 1.5608 Acc: 0.4214
Val   Loss: 1.3841 Acc: 0.4817
✅ Best model saved.
Epoch 4/20
Train Loss: 1.4433 Acc: 0.4584
Val   Loss: 1.3427 Acc: 0.4879
✅ Best model saved.
Epoch 5/20
Train Loss: 1.3785 Acc: 0.4869
Val   Loss: 1.2341 Acc: 0.5410
✅ Best model saved.
Epoch 6/20
Train Loss: 1.3346 Acc: 0.5030
Val   Loss: 1.3480 Acc: 0.5011
Epoch 7/20
Train Loss: 1.2839 Acc: 0.5169
Val   Loss: 1.3982 Acc: 0.4902
Epoch 8/20
Train Loss: 1.2806 Acc: 0.5196
Val   Loss: 1.3144 Acc: 0.5258
Epoch 9/20
Train Loss: 1.2507 Acc: 0.5275
Val   Loss: 1.2062 Acc: 0.5436
✅ Best model saved.
Epoch 10/20
Train Loss: 1.2286 Acc: 0.5361
Val   Loss: 1.2244 Acc: 0.5412
Epoch 11/20
Train Loss: 1.1880 Acc: 0.5472
Val   Loss: 1.2110 Acc: 0.5508
✅ Best model saved.
Epoch 12/20
Train Loss: 1.1680 Acc: 0.5582
Val   Loss: 